- epoch 200 and es 50 
- incorporate adamw
- recutangular padding 
- cancel cutmix, mixup 
- check oof score * 
- try vit from scratch * 
- cosine annealing lr scheduler *

In [1]:
!pip install ttach

In [2]:
!pip install lightly

     |████████████████████████████████| 256 kB 2.0 MB/s 
     |████████████████████████████████| 144 kB 13.8 MB/s 
     |████████████████████████████████| 74 kB 2.9 MB/s 
     |████████████████████████████████| 112 kB 13.8 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=eebbbaa06160ed67aa166a23f9bf1a439982db62d70d8317d9bafb431318ae21
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3-runtime


In [3]:
!pip install timm

     |████████████████████████████████| 376 kB 2.1 MB/s 


In [4]:
import os
import sys
import cv2
import time
import math
import timm
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from pathlib import Path
from collections import OrderedDict
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold

import torch
from torch import nn
import tensorflow as tf
from torch.utils import data
import torch.nn.functional as F
from torch.optim import Adam, AdamW
from torch.optim.optimizer import Optimizer
from collections import defaultdict, Counter, OrderedDict
from torch.optim.lr_scheduler import OneCycleLR, ReduceLROnPlateau, CosineAnnealingWarmRestarts,  CosineAnnealingLR

# various models can be selected: https://pytorch.org/vision/stable/models.html
import torchvision
from torchvision import transforms as T
from torchvision.models import resnet34, resnet18

import ttach as tta
import lightly

# pytorch-lightning
import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.plugins import DDPPlugin
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.loggers.csv_logs import CSVLogger
from pytorch_lightning.callbacks.progress import ProgressBarBase
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks import LearningRateMonitor

In [5]:
n_folds = 5
MAX_EPOCHS = 200
ES_PATIENCE = 50
IS_DEBUG = False
BATCH_SIZE = 64
NUM_WORKERS = 10
mixup_alpha = 0.4
cutmix_alpha = 0.4
mixup_freq = 0.8

dataset_root = "../input/atma11-dataset/"
photo_dir = "../input/atma11-dataset/photos/"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 基本構成
- class MyLightningModule(LightningModule):
 決めてある関数の中身を埋める。各々クラス内関数を定義する

- class MyDataModule(LightningDataModule):
 決めてある関数の中身を埋める。各々クラス内関数を定義する

- trainer = Trainer()
- lightning_module = MyLightningModule()
- data_module = MyDataModule()

- trainer.fit(lightning_module, datamodule=data_module) 学習開始

# read data

In [6]:
train_df = pd.read_csv("../input/atma11-dataset/train.csv")
test_df = pd.read_csv("../input/atma11-dataset/test.csv")

# functions

In [7]:
def to_img_path(object_id):
    return os.path.join(photo_dir, f'{object_id}.jpg')

def read_image(object_id):
    return Image.open(to_img_path(object_id))

def calculate_metrics(y_true, y_pred) -> dict:
    return {'rmse': mean_squared_error(y_true, y_pred) ** .5}

def create_metadata(input_df):
    out_df = input_df[['object_id']].copy()
    out_df['object_path'] = input_df['object_id'].map(to_img_path)

    if "target" in input_df:
        out_df["target"] = input_df["target"]

    return out_df

In [8]:
def stratified_group_k_fold(X, y, groups, k, seed=None):
    labels_num = np.max(y) + 1
    y_counts_per_group = defaultdict(lambda: np.zeros(labels_num))
    y_distr = Counter()
    for label, g in zip(y, groups):
        y_counts_per_group[g][label] += 1
        y_distr[label] += 1

    y_counts_per_fold = defaultdict(lambda: np.zeros(labels_num))
    groups_per_fold = defaultdict(set)

    def eval_y_counts_per_fold(y_counts, fold):
        y_counts_per_fold[fold] += y_counts
        std_per_label = []
        for label in range(labels_num):
            label_std = np.std([y_counts_per_fold[i][label] / y_distr[label] for i in range(k)])
            std_per_label.append(label_std)
        y_counts_per_fold[fold] -= y_counts
        return np.mean(std_per_label)
    
    groups_and_y_counts = list(y_counts_per_group.items())
    random.Random(seed).shuffle(groups_and_y_counts)

    for g, y_counts in sorted(groups_and_y_counts, key=lambda x: -np.std(x[1])):
        best_fold = None
        min_eval = None
        for i in range(k):
            fold_eval = eval_y_counts_per_fold(y_counts, i)
            if min_eval is None or fold_eval < min_eval:
                min_eval = fold_eval
                best_fold = i
        y_counts_per_fold[best_fold] += y_counts
        groups_per_fold[best_fold].add(g)

    all_groups = set(groups)
    for i in range(k):
        train_groups = all_groups - groups_per_fold[i]
        test_groups = groups_per_fold[i]

        train_indices = [i for i, g in enumerate(groups) if g in train_groups]
        test_indices = [i for i, g in enumerate(groups) if g in test_groups]

        yield train_indices, test_indices

In [9]:
# cutmix, mixup 
# https://github.com/ishikei14k/atma11_1st_solution/blob/161c0851fa6b97c431ea1b34788b18938d730ccf/src/utils/cutmix_mixup.py
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

def cutmix(data, targets, alpha):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]
    #for target in targets:
    #    shuffled_targets.append(targets[indices])

    lam = np.random.beta(alpha, alpha)
    bbx1, bby1, bbx2, bby2 = rand_bbox(data.size(), lam)
    data[:, :, bbx1:bbx2, bby1:bby2] = data[indices, :, bbx1:bbx2, bby1:bby2]
    # adjust lambda to exactly match pixel ratio
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (data.size()[-1] * data.size()[-2]))

    results = {
        'targets': targets,
        'shuffled_targets': shuffled_targets,
        'lam': lam
    }
    return data, results

def mixup(data, targets, alpha):
    """
    original mixup.
    """
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]

    lam = np.random.beta(alpha, alpha)
    data = data * lam + shuffled_data * (1 - lam)

    results = {
        'targets': targets,
        'shuffled_targets': shuffled_targets,
        'lam': lam
    }
    return data, results


def mixup2(data, targets, alpha):
    """
    mixup: use different lambda each samples.
    """
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]

    lams = [np.random.beta(alpha, alpha) for i in range(len(data))]
    #print(lams)

    for i in range(len(data)):
        data[i] = data[i] * lams[i] + shuffled_data[i] * (1 - lams[i])

    results = {
        'targets': targets,
        'shuffled_targets': shuffled_targets,
        'lam': lams
    }
    return data, results

def mixup_3sample(data, targets, alpha):
    """
    mixup: mixup 3 sample.
    """
    indices1 = torch.randperm(data.size(0))
    indices2 = torch.randperm(data.size(0))
    
    shuffled_data1 = data[indices1]
    shuffled_targets1 = targets[indices1]
    
    shuffled_data2 = data[indices2]
    shuffled_targets2 = targets[indices2]

    lam0, lam1, lam2 = np.random.dirichlet([alpha, alpha, alpha])
    data = data * lam0 + shuffled_data1 * lam1 + shuffled_data2 * lam2

    results = {
        'targets': targets,
        'shuffled_targets1': shuffled_targets1,
        'shuffled_targets2': shuffled_targets2,
        'lam': [lam0, lam1, lam2]
    }
    return data, results


def cutmix_soft_label(data, targets, alpha):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = []
    for target in targets:
        shuffled_targets.append(target[indices])

    lam = np.random.beta(alpha, alpha)
    bbx1, bby1, bbx2, bby2 = rand_bbox(data.size(), lam)
    data[:, :, bbx1:bbx2, bby1:bby2] = data[indices, :, bbx1:bbx2, bby1:bby2]
    # adjust lambda to exactly match pixel ratio
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (data.size()[-1] * data.size()[-2]))

    results = {
        'targets': targets,
        'shuffled_targets': shuffled_targets,
        'lam': lam
    }
    return data, results

def mixup_soft_label(data, targets, alpha):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = []
    for target in targets:
        shuffled_targets.append(target[indices])

    lam = np.random.beta(alpha, alpha)
    data = data * lam + shuffled_data * (1 - lam)

    results = {
        'targets': targets,
        'shuffled_targets': shuffled_targets,
        'lam': lam
    }
    return data, results

In [10]:
class NotebookProgress(ProgressBarBase):

    def __init__(self):
        super().__init__()  # don't forget this :)
        self.enable = True

    def disable(self):
        self.enable = False

    def on_train_batch_end(self, trainer, pl_module, outputs, batch, batch_idx, dataloader_idx):
        super().on_train_batch_end(trainer, pl_module, outputs, batch, batch_idx, dataloader_idx)  # don't forget this :)
        percent = (self.train_batch_idx / self.total_train_batches) * 100
        sys.stdout.flush()
        sys.stdout.write(f'{percent:.01f} percent complete \r')

    def on_validation_batch_end(self, trainer, pl_module, outputs, batch, batch_idx, dataloader_idx):
        super().on_validation_batch_end(trainer, pl_module, outputs, batch, batch_idx, dataloader_idx)# don't forget this :)

# data module

In [11]:
class AtmaDataset(data.Dataset):
    """atmaCup用にデータ読み込み等を行なうデータ・セット"""
    object_path_key = "object_path"
    target_key = "target"

    @property
    def meta_keys(self):
        """"""
        retval = [self.object_path_key]

        if self.phase == 'train' or self.phase == 'valid':
            retval.append(self.target_key)

        return retval

    def __init__(self, meta_df: pd.DataFrame, phase='train', transform=None):
        """
        args:
            meta_df: 
                画像へのパスと label 情報が含まれている dataframe
                必ず object_path に画像へのパス, target に正解ラベルが入っている必要があります

            phase:
                train のとき学習用のデータ拡張を適用します.
                train 以外の時は単に size にリサイズを行います

            transform:
                画像にかける変換処理
        """

        self.phase = phase
        for k in self.meta_keys:
            if k not in meta_df:
                raise ValueError(f"meta_df must have {k}")

        self.meta_df = meta_df.reset_index(drop=True)
        self.index_to_data = self.meta_df.to_dict(orient="index")
        self.transform = transform

        size = (224, 224)
        
    def resize_fill(self, _img):
        _img = np.array(_img)
        w,h,c = _img.shape
        size = (224, 224)
        if w>h:
            left = int((w-h)/2)
            right = int((w-h)/2)
            top = 0
            bottom = 0
        elif h>w:
            top = int((h-w)/2)
            bottom = int((h-w)/2)
            left = 0
            right = 0
        else:
            left = 0
            right = 0
            top = 0
            bottom = 0

        _img = cv2.copyMakeBorder(
            _img,
            top=top,
            bottom=bottom,
            left=left,
            right=right,
            borderType=cv2.BORDER_REPLICATE,
            value=[0,0,0]
            )
        
        #_img = cv2.resize(_img, size)
        return  Image.fromarray(np.uint8(_img))

    def __getitem__(self, index):
        data = self.index_to_data[index]

        obj_path, target = (data.get(self.object_path_key),
                            data.get(self.target_key, -1),)
        img = Image.open(obj_path)
        img = self.resize_fill(img) # image padding to rectangular
        img = self.transform(img)

        if self.phase == 'test':
            return img

        return img, target

    def __len__(self):
        return len(self.meta_df)

In [12]:
class AtmaDataModule(pl.LightningDataModule):
    def __init__(self, dataset_root: str = "../input",
                    batch_size: int = 32,
                    fold: int = 0,
                    num_workers: int = 20,):
        super().__init__()
        self.dataset_root = dataset_root 
        self.batch_size = batch_size 
        self.fold = fold 
        self.num_workers = num_workers
        
    def get_test_df(self):
        return pd.read_csv(os.path.join(self.dataset_root, 'test.csv'))
    
    def split_train_valid_df(self):
        df = pd.read_csv(os.path.join(self.dataset_root, 'train.csv'))
        groups = df["art_series_id"].values
        
        sgkf = stratified_group_k_fold(df, df['target'], groups, k=n_folds, seed=12)
        for i, (train_index, val_index) in enumerate(sgkf):
            df.loc[val_index, "fold"] = int(i)
        df["fold"] = df["fold"].astype(int)

        train_df = df[df["fold"] != self.fold].reset_index(drop=True)
        valid_df = df[df["fold"] == self.fold].reset_index(drop=True)
        return train_df, valid_df

    # 必ず呼び出される関数
    def setup(self, stage=None):
        self.test_df = self.get_test_df()
        train_df, valid_df = self.split_train_valid_df()
        self.train_df = train_df
        self.valid_df = valid_df
        
    def get_dataframe(self, phase):
        assert phase in {"train", "valid", "test"}
        if phase == "train":
            return self.train_df
        elif phase == "valid":
            return self.valid_df
        elif phase == "test":
            return self.test_df
                
    def get_transform(self, phase):
        IMG_MEAN = [0.485, 0.456, 0.406]
        IMG_STD = [0.229, 0.224, 0.225]
        size = (224, 224)

        additional_items = (
            [T.Resize(size)] 
            if phase != 'train'
            else [
                T.RandomGrayscale(p=0.2),
                T.RandomVerticalFlip(),
                T.RandomHorizontalFlip(),
                T.RandomResizedCrop(size),
            ]
        )

        transform = T.Compose(
            [*additional_items, 
             T.ToTensor(), 
             T.Normalize(mean=IMG_MEAN, std=IMG_STD),
            ])

        return transform

    def get_ds(self, phase):
        assert phase in {"train", "valid", "test"}
        ds = AtmaDataset(
            meta_df=create_metadata(self.get_dataframe(phase=phase)),
            transform=self.get_transform(phase=phase),
            phase=phase,
        )
        return ds
    
    def get_loader(self, phase):
        dataset = self.get_ds(phase=phase)

        return data.DataLoader(
            dataset,
            batch_size=self.batch_size,
            shuffle=True if phase == "train" else False,
            num_workers=self.num_workers,
            drop_last=True if phase == "train" else False,
        )

    def train_dataloader(self):
        return self.get_loader(phase='train')

    def val_dataloader(self):
        return self.get_loader(phase='valid')

    def predict_dataloader(self):
        return self.get_loader(phase='test')

# lightning module

In [13]:
class LitModule(pl.LightningModule):

    def get_model(self):
        # resnet18d
        #model = resnet18(pretrained=False)
        #model.fc = nn.Linear(in_features=512, out_features=1, bias=True)
        
        # vit
        model = timm.create_model('vit_tiny_r_s16_p8_224', pretrained=False)
        model.head = nn.Linear(in_features=192, out_features=1, bias=True)
        return model
    
    def get_crit(self):
        return nn.MSELoss()
    
    def calculate_elapsed_time(self):
        elapsed_time = int(time.time() - self.start_time)
        elapsed_minute = (elapsed_time % 3600) / 60
        return elapsed_minute
    
    def __init__(self):
        super(LitModule, self).__init__()
        self.net = self.get_model()
        self.crit = self.get_crit()
        self.start_time = time.time()
        
    def forward(self, x):
        output = self.net(x).squeeze(1)
        return output
        
    def training_step(self, batch, batch_idx):
        imgs, targets = batch
#         p = np.random.rand()
        
#         if 0.0<=p<mixup_freq/2:
#         # apply mixup.
#             imgs, results = mixup(imgs, targets, mixup_alpha)
#             preds = self(imgs)
#             #loss calc
#             targets = results['targets']
#             shuffled_targets = results['shuffled_targets']
#             lam = results['lam']
#             loss = lam*self.crit(preds, targets.float()) + (1-lam)*self.crit(preds, shuffled_targets.float())
        
#         elif mixup_freq/2<=p<mixup_freq: # note use.
#         # apply cutmix.
#             imgs, results = cutmix(imgs, targets, cutmix_alpha)
#             preds = self(imgs)
#             #loss calc
#             targets = results['targets']
#             shuffled_targets = results['shuffled_targets']
#             lam = results['lam']
#             loss = lam*self.crit(preds, targets.float()) + (1-lam)*self.crit(preds, shuffled_targets.float())

#         else: # wo mixup cutmix. (original)     
        output = self(imgs)
        loss = self.crit(output, targets.float())
            
        return loss

    def training_epoch_end(self, outputs):
        train_epoch_loss = torch.stack([o["loss"] for o in outputs]).mean().item()
        self.log('train_loss', train_epoch_loss)
        print(f'train_epoch_loss : {train_epoch_loss}')

    def validation_step(self, batch, batch_idx):
        imgs, targets = batch
        preds = self(imgs)
        loss = self.crit(preds, targets.float())

        output = OrderedDict({
            "preds": preds.detach(),
            "targets": targets.detach(),
            "loss": loss.detach(),
        })
        return output

    def validation_epoch_end(self, outputs):
        d = dict()
        d["epoch"] = int(self.current_epoch)
        d["valid_loss"] = round(torch.stack([o["loss"] for o in outputs]).mean().item(), 8)

        preds = torch.cat([o["preds"].view(-1) for o in outputs]).cpu().numpy()
        targets = torch.cat([o["targets"].view(-1) for o in outputs]).cpu().numpy()

        rmse_target = np.sqrt(mean_squared_error(y_true=targets, y_pred=preds))
        d["valid_rmse"] = round(rmse_target, 8)

        d["time(m)"] = round(self.calculate_elapsed_time(), 3)

        self.log_dict(d)
        print(d)

    def configure_optimizers(self):
        optimizer = AdamW(self.net.parameters(), lr=1e-5, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=False)
        
        lr_scheduler = {
            'scheduler': CosineAnnealingLR(optimizer, T_max = MAX_EPOCHS),
            #CosineAnnealingWarmRestarts(optimizer, T_0 = 5, T_mult=1, eta_min=1e-4, 
            #                                        last_epoch=-1, verbose=False),
            'monitor': 'valid_loss',
        }

        return [optimizer], [lr_scheduler]

# main training and validation

In [14]:
def train(cfg_name: str, output_path: str, fold: int,) -> None:
    seed_everything(42)

    logger = CSVLogger(save_dir=output_path, name=f"fold_{fold}")

    # 学習済重みを保存するために必要
    checkpoint_callback = ModelCheckpoint(
        dirpath=str(output_path), filename=f"{cfg_name}_{fold}",
        save_weights_only=True,
        save_top_k=None,
        monitor='valid_rmse',
    )

    early_stop_callback = EarlyStopping(
        monitor='valid_rmse',
        min_delta=0.00,
        patience=ES_PATIENCE,
        verbose=False,
        mode='min'
    )

    progress = NotebookProgress()
    lr_monitor = LearningRateMonitor(logging_interval='epoch')

    trainer = Trainer(
        max_epochs=MAX_EPOCHS,
        gpus=1,
        callbacks=[checkpoint_callback, 
                   early_stop_callback,
                   progress,
                   lr_monitor,],
        logger=[logger],
        limit_train_batches=0.02 if IS_DEBUG else 1.0,
        limit_val_batches=0.05 if IS_DEBUG else 1.0,
    )

    datamodule = AtmaDataModule(dataset_root=dataset_root,
                                batch_size=BATCH_SIZE,
                                fold = fold,
                                num_workers=NUM_WORKERS)
    
    model = LitModule()
    trainer.fit(model, datamodule=datamodule)

    return checkpoint_callback.best_model_path, checkpoint_callback.best_model_score

In [15]:
output_path = '../output_pl_first'
os.makedirs(output_path, exist_ok=True)
best_model_paths = []
oof = np.zeros((len(train_df), ), dtype=np.float32)

sum_mse = 0
for fold in tqdm(range(n_folds)):
    best_model_path, best_model_score = train(cfg_name=f'pl_first', output_path=output_path, fold=fold)
    best_model_paths.append(best_model_path)

    sum_mse += best_model_score / n_folds
    print(f'best_model_path : {best_model_path}')
    print(f'best_model_score : {best_model_score}')

# CVスコアを表示
print("overall score: ", sum_mse ** 0.5)

  0%|          | 0/5 [00:00<?, ?it/s]

{'epoch': 0, 'valid_loss': 2.76468706, 'valid_rmse': 1.66273484, 'time(m)': 0.183}
train_epoch_loss : 1.0665653944015503
{'epoch': 0, 'valid_loss': 0.92302823, 'valid_rmse': 0.95274787, 'time(m)': 0.533}
train_epoch_loss : 0.9127307534217834
{'epoch': 1, 'valid_loss': 0.91324663, 'valid_rmse': 0.94774427, 'time(m)': 0.883}
train_epoch_loss : 0.9104193449020386
{'epoch': 2, 'valid_loss': 0.90532351, 'valid_rmse': 0.94364623, 'time(m)': 1.233}
train_epoch_loss : 0.8907366991043091
{'epoch': 3, 'valid_loss': 0.90111023, 'valid_rmse': 0.94083302, 'time(m)': 1.583}
train_epoch_loss : 0.8857883214950562
{'epoch': 4, 'valid_loss': 0.93936467, 'valid_rmse': 0.96130936, 'time(m)': 1.933}
train_epoch_loss : 0.8961957097053528
{'epoch': 5, 'valid_loss': 0.89464748, 'valid_rmse': 0.93634795, 'time(m)': 2.267}
train_epoch_loss : 0.8846265077590942
{'epoch': 6, 'valid_loss': 0.89466465, 'valid_rmse': 0.93583777, 'time(m)': 2.617}
train_epoch_loss : 0.8887335062026978
{'epoch': 7, 'valid_loss': 0.881

 20%|██        | 1/5 [57:01<3:48:06, 3421.52s/it]

{'epoch': 167, 'valid_loss': 0.73524278, 'valid_rmse': 0.84256189, 'time(m)': 57.017}
best_model_path : /kaggle/output_pl_first/pl_first_0.ckpt
best_model_score : 0.8318028450012207
{'epoch': 0, 'valid_loss': 3.14661551, 'valid_rmse': 1.77387014, 'time(m)': 0.083}
train_epoch_loss : 0.9966296553611755
{'epoch': 0, 'valid_loss': 0.93101227, 'valid_rmse': 0.95877225, 'time(m)': 0.45}
train_epoch_loss : 0.8944063782691956
{'epoch': 1, 'valid_loss': 0.93471509, 'valid_rmse': 0.96165896, 'time(m)': 0.783}
train_epoch_loss : 0.8850234746932983
{'epoch': 2, 'valid_loss': 0.92963713, 'valid_rmse': 0.95440619, 'time(m)': 1.133}
train_epoch_loss : 0.8869143128395081
{'epoch': 3, 'valid_loss': 0.9219498, 'valid_rmse': 0.95144693, 'time(m)': 1.483}
train_epoch_loss : 0.8834602236747742
{'epoch': 4, 'valid_loss': 0.96254528, 'valid_rmse': 0.96726946, 'time(m)': 1.833}
train_epoch_loss : 0.88609379529953
{'epoch': 5, 'valid_loss': 0.9749704, 'valid_rmse': 0.97309034, 'time(m)': 2.183}
train_epoch_lo

 40%|████      | 2/5 [1:40:34<2:27:16, 2945.67s/it]

...

{'epoch': 199, 'valid_loss': 0.67987657, 'valid_rmse': 0.82505871, 'time(m)': 8.183}
best_model_path : /kaggle/output_pl_first/pl_first_2.ckpt
best_model_score : 0.8196853399276733
{'epoch': 0, 'valid_loss': 3.1888845, 'valid_rmse': 1.78574479, 'time(m)': 0.083}
train_epoch_loss : 1.0215445756912231
{'epoch': 0, 'valid_loss': 0.89851648, 'valid_rmse': 0.94643579, 'time(m)': 0.433}
train_epoch_loss : 0.9095095992088318
{'epoch': 1, 'valid_loss': 0.89092869, 'valid_rmse': 0.94549588, 'time(m)': 0.767}
train_epoch_loss : 0.9072487950325012
{'epoch': 2, 'valid_loss': 0.88743353, 'valid_rmse': 0.94041971, 'time(m)': 1.1}
train_epoch_loss : 0.907148540019989
{'epoch': 3, 'valid_loss': 0.8918224, 'valid_rmse': 0.94629307, 'time(m)': 1.45}
train_epoch_loss : 0.9038218259811401
{'epoch': 4, 'valid_loss': 0.86791146, 'valid_rmse': 0.93113888, 'time(m)': 1.783}
train_epoch_loss : 0.8838254809379578
{'epoch': 5, 'valid_loss': 0.85761255, 'valid_rmse': 0.92532957, 'time(m)': 2.15}
train_epoch_loss 

 80%|████████  | 4/5 [3:58:17<1:02:26, 3746.21s/it]

{'epoch': 199, 'valid_loss': 0.66542435, 'valid_rmse': 0.8182384, 'time(m)': 9.5}
best_model_path : /kaggle/output_pl_first/pl_first_3.ckpt
best_model_score : 0.8170464038848877
{'epoch': 0, 'valid_loss': 2.91970348, 'valid_rmse': 1.70871394, 'time(m)': 0.083}
train_epoch_loss : 1.0175551176071167
{'epoch': 0, 'valid_loss': 0.91135657, 'valid_rmse': 0.94879887, 'time(m)': 0.433}
train_epoch_loss : 0.9113854169845581
{'epoch': 1, 'valid_loss': 0.91519111, 'valid_rmse': 0.95215037, 'time(m)': 0.767}
train_epoch_loss : 0.9179884791374207
{'epoch': 2, 'valid_loss': 0.91322035, 'valid_rmse': 0.95137902, 'time(m)': 1.133}
train_epoch_loss : 0.8946186304092407
{'epoch': 3, 'valid_loss': 0.89693344, 'valid_rmse': 0.94003164, 'time(m)': 1.467}
train_epoch_loss : 0.8880982398986816
{'epoch': 4, 'valid_loss': 0.87836409, 'valid_rmse': 0.93231595, 'time(m)': 1.817}
train_epoch_loss : 0.9066806435585022
{'epoch': 5, 'valid_loss': 0.89545482, 'valid_rmse': 0.93942346, 'time(m)': 2.15}
train_epoch_lo

100%|██████████| 5/5 [5:07:13<00:00, 3686.74s/it]

{'epoch': 197, 'valid_loss': 0.68638366, 'valid_rmse': 0.82324123, 'time(m)': 8.933}
best_model_path : /kaggle/output_pl_first/pl_first_4.ckpt
best_model_score : 0.8203914761543274
overall score:  tensor(0.9101, device='cuda:0')


# oof

In [16]:
def load_pytorch_model(state_dict, *args, **kwargs):
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k
        if name.startswith('net.'):
            name = name.replace('net.', '') # remove `net.`
        new_state_dict[name] = v
    
    #resnet
    #model = resnet18(pretrained=False)
    #model.fc = nn.Linear(in_features=512, out_features=1, bias=True)
    
    # vit
    model = timm.create_model('vit_tiny_r_s16_p8_224', pretrained=False)
    model.head = nn.Linear(in_features=192, out_features=1, bias=True)
    
    model.load_state_dict(new_state_dict)
    return model

def predict(model, dataloader, device, tta=1):
    model.eval()
    model.to(device)
    preds = np.array([])
    for data, _ in dataloader:
        data = data.to(device)
        with torch.no_grad():
            y_pred = model(data).detach()
        y_pred = y_pred.cpu().numpy()
        y_pred = y_pred.reshape(-1)
        preds = np.concatenate([preds, y_pred])
    return preds

In [17]:
oof = np.zeros((len(train_df), ), dtype=np.float32)
groups = train_df["art_series_id"].values
sgkf = stratified_group_k_fold(train_df, train_df['target'], groups, k=n_folds, seed=12)
transform = AtmaDataModule().get_transform(phase="valid")

for i, (train_index, val_index) in enumerate(sgkf):
    model_path = best_model_paths[i]
    ckpt_dict = torch.load(model_path)
    best_model = load_pytorch_model(ckpt_dict['state_dict'])

    valid_df = train_df.iloc[val_index].copy()
    valid_df = create_metadata(valid_df)
    valid_dataset = AtmaDataset(meta_df = valid_df, phase='valid', transform=transform)
    
    val_loader = data.DataLoader(valid_dataset, batch_size=256, num_workers=4)
    oof[val_index] = predict(best_model, val_loader, device=device)

In [18]:
oof = np.clip(oof, 0, 3)
score = np.sqrt(mean_squared_error(train_df['target'], oof))
score

0.8280926157941131

# prediction

In [19]:
predictions = np.zeros(len(test_df))

for model_path in best_model_paths:
    model = LitModule().load_from_checkpoint(checkpoint_path=model_path)
    trainer = Trainer(gpus=1)

    datamodule = AtmaDataModule(dataset_root=dataset_root,
                                batch_size=BATCH_SIZE,
                                num_workers=NUM_WORKERS)

    preds = trainer.predict(model, datamodule=datamodule, return_predictions=True)
    preds = torch.cat(preds)
    predictions += preds.cpu().numpy() / n_folds

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

In [20]:
predictions = np.clip(predictions, 0, 3)

# submission

In [21]:
pd.DataFrame({"target": predictions}).to_csv("vit_sub"+str(score)+".csv", index=False)
pd.DataFrame({"oof": oof}).to_csv("vit_oof"+str(score)+".csv", index=False)